# Sesión 1 – Apartado 2: Limpieza y preprocesamiento avanzado

**Duración recomendada:** ~80 min

**Objetivos:**
- Identificar y tratar valores nulos.
- Aplicar diferentes técnicas de imputación.
- Codificar variables categóricas.
- Escalar y normalizar variables numéricas.


## 1) Cargar el dataset sintético

In [ ]:
import pandas as pd
df = pd.read_csv('data/mini_titanic.csv')
df.head()

## 2) Identificación de valores nulos

In [ ]:
df.isna().sum().sort_values(ascending=False)

## 3) Imputación de valores nulos

Probamos varias estrategias de imputación:
- Media (para variables numéricas)
- Mediana
- KNNImputer (más avanzado)


In [ ]:
from sklearn.impute import SimpleImputer, KNNImputer
import numpy as np

# Imputación con media
imputer_mean = SimpleImputer(strategy='mean')
df['age_imputed'] = imputer_mean.fit_transform(df[['age']])

# Imputación con mediana
imputer_median = SimpleImputer(strategy='median')
df['fare_imputed'] = imputer_median.fit_transform(df[['fare']])

# KNNImputer en conjunto numérico
num_cols = ['age', 'fare']
knn_imputer = KNNImputer(n_neighbors=5)
df_knn = pd.DataFrame(knn_imputer.fit_transform(df[num_cols]), columns=num_cols)
df_knn.head()

## 4) Codificación de variables categóricas

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# OneHotEncoder en columna 'sex'
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
sex_encoded = encoder.fit_transform(df[['sex']])
sex_encoded[:5]

In [ ]:
# OrdinalEncoder en columna 'class'
ord_enc = OrdinalEncoder()
df['class_encoded'] = ord_enc.fit_transform(df[['class']])
df[['class', 'class_encoded']].head()

## 5) Escalado y normalización

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

scaler_std = StandardScaler()
scaler_minmax = MinMaxScaler()
scaler_robust = RobustScaler()

age_std = scaler_std.fit_transform(df[['age_imputed']])
age_minmax = scaler_minmax.fit_transform(df[['age_imputed']])
age_robust = scaler_robust.fit_transform(df[['age_imputed']])

import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))
plt.hist(age_std, bins=20, alpha=0.5, label='StandardScaler')
plt.hist(age_minmax, bins=20, alpha=0.5, label='MinMaxScaler')
plt.hist(age_robust, bins=20, alpha=0.5, label='RobustScaler')
plt.legend()
plt.title('Distribución de Edad con diferentes escalados')
plt.show()

## 6) Ejercicios

1. Imputa la columna `fare` usando KNNImputer y compárala con la imputación por mediana.
2. Codifica la columna `class` usando OneHotEncoder en lugar de OrdinalEncoder. ¿Qué diferencias observas?
3. Escala la variable `fare` con los tres métodos (Standard, MinMax, Robust) y grafica los histogramas.
4. Discute: ¿qué método de imputación y escalado usarías para este dataset y por qué?
